# Trabalho desenvolvido na matéria de programação linear
### Estevão Nunes da Silva, Hugo Lima Otoch, João Gabriel Barros, Tales Viana



# Otimização de Seleção de Jogadores no Cartola FC utilizando Programação Matemática

A otimização da formação de equipes em esportes é um problema complexo que envolve a combinação de análise de dados e técnicas de otimização para selecionar um conjunto ideal de jogadores. Este problema, conhecido como Team Formation Problem (TFP), é amplamente estudado no campo da pesquisa operacional e tem implicações práticas significativas para gerentes de equipes e analistas de desempenho esportivo. Neste notebook, utilizamos a API do Cartola FC para obter dados atualizados dos jogadores e empregamos técnicas de otimização usando Gurobi para formar uma equipe de futebol ideal com base em critérios específicos, como orçamento e posição dos jogadores.

De acordo com [Muniz (2022)](https://repository.mines.edu/bitstream/handle/11124/15458/Muniz_mines_0052E_12420.pdf?sequence=1), a integração de técnicas de otimização e análise esportiva proporciona uma abordagem sistemática para a tomada de decisões em contextos desportivos. A abordagem utilizada nesta () documentação baseia-se em metodologias desenvolvidas no contexto da NBA, mas pode ser generalizada para outros esportes, incluindo o futebol.




*   [MUNIZ, Megan L. Sports Analytics and Optimization for Team Formation Problems. 2022. Tese de Doutorado. Colorado School of Mines.](https://repository.mines.edu/bitstream/handle/11124/15458/Muniz_mines_0052E_12420.pdf?sequence=1)
*   O conceito de Moneyball, popularizado pelo livro de Michael Lewis e o filme subsequente, demonstra como a análise de dados pode transformar a formação de equipes esportivas, focando em estatísticas subvalorizadas para otimizar a performance e maximizar o desempenho de equipes com orçamentos limitados. Esta abordagem inovadora foi inicialmente implementada pelo Oakland Athletics na MLB, mas seus princípios se espalharam por vários esportes, incluindo o futebol, onde ferramentas de otimização e análise de dados, como as utilizadas no Cartola FC, têm o potencial de influenciar a tomada de decisões estratégicas .

A combinação de técnicas de otimização como o Gurobi com análise de desempenho esportivo segue a mesma linha de pensamento do Moneyball, onde a maximização da eficiência e do valor dos jogadores se torna o foco central na formação de equipes. [Moneyball](https://pt.wikipedia.org/wiki/Moneyball)




## Cartola FC

O *Cartola FC* é um jogo virtual de fantasy football muito popular no Brasil, no qual os jogadores formam equipes virtuais com base em atletas reais que atuam no Campeonato Brasileiro de Futebol. Cada jogador de futebol do mundo real recebe uma pontuação no *Cartola FC* de acordo com seu desempenho em partidas reais, com base em critérios como gols, assistências, defesas, faltas cometidas e sofridas, cartões, entre outros.

No Cartila o objetivo principal é formar a equipe mais eficiente possível para cada rodada do Campeonato Brasileiro. Cada competidor seleciona 11 jogadores, que podem pertencer a diferentes times do mundo real, formando assim um "time" virtual. A pontuação do time é baseada no desempenho individual de cada jogador em partidas reais daquela rodada. No entanto, o desempenho de um jogador não depende apenas de suas habilidades, mas também do adversário que enfrenta.

Por exemplo, um zagueiro de alto nível pode pontuar mal em uma rodada se estiver enfrentando um time com ataque poderoso. Por outro lado, um defensor de desempenho mediano pode se sair bem se enfrentar um time com um ataque fraco. Esse aspecto estratégico torna o Cartola FC único, pois o desafio não é apenas escolher os melhores jogadores, mas também analisar o contexto dos confrontos da rodada para maximizar a pontuação.

Para compreender melhor como montar um time competitivo, é essencial explicar: (a) a formação tática permitida pelas regras do Cartola FC e (b) os critérios de pontuação individuais para cada jogador. Esses fatores são cruciais para formular uma estratégia vencedora.


### Formação 4-3-3
No contexto deste estudo, utilizamos a formação tática 4-3-3 como uma restrição no problema de otimização. Essa formação é amplamente adotada tanto no *Cartola FC* quanto no futebol profissional. Ela consiste em quatro defensores (normalmente dois zagueiros e dois laterais), três meio-campistas e três atacantes. A escolha de restringir o problema a essa formação específica visa simplificar o processo de otimização, ao mesmo tempo em que mantém uma estrutura tática equilibrada, comum nas regras do jogo.


### Pontuação do Jogador no *Cartola FC*
A pontuação de cada jogador no *Cartola FC* reflete o desempenho do atleta em campo, sendo calculada com base em uma série de métricas de desempenho individuais. Exemplos de métricas incluem:

- **Ações positivas**: Gols, assistências, desarmes, defesas difíceis (para goleiros), passes decisivos.
- **Ações negativas**: Cartões amarelo/vermelho, faltas cometidas, passes errados, perda de pênaltis.

Cada uma dessas ações possui um peso específico, e o somatório de todas elas resulta na pontuação final do jogador em uma rodada. Esse sistema de pontuação é o que orienta as decisões estratégicas dos jogadores de *Cartola FC*, ao formar suas equipes, já que é fundamental escalar jogadores com potencial de alta pontuação para maximizar o desempenho geral da equipe.

A utilização dessas métricas dentro de técnicas de otimização, como o Gurobi, permite que uma equipe virtual seja formada com base em critérios quantitativos rigorosos, levando em consideração o orçamento disponível e as posições táticas obrigatórias, como a formação 4-3-3.


### Escolha da Função Objetivo

Ao formular o problema de otimização para formar uma equipe ideal no *Cartola FC*, é essencial definir uma função objetivo que oriente a seleção dos jogadores, em cada rodada, de maneira a maximizar o desempenho da equipe. Diversas funções objetivo podem ser adotadas, dependendo dos critérios de análise e das metas estabelecidas para o time. 

Neste trabalho, optamos por utilizar a **maximização da pontuação média dos jogadores selecionados** como função objetivo principal. Isso significa que a seleção da equipe visa obter a maior pontuação possível com base na média de pontuação histórica de cada jogador. Esse enfoque é simples e direto, considerando o desempenho geral dos jogadores em múltiplos jogos.

No entanto, outras opções de função objetivo poderiam ser adotadas, como:

- **Maximização de características específicas por posição**:
  - **Defensores**: Priorizar jogadores com maior número de desarmes, interceptações e jogos sem sofrer gols.
  - **Meio-campistas**: Focar em assistências, chutes a gol e participação em jogadas ofensivas.
  - **Atacantes**: Dar prioridade a gols marcados, chutes a gol e participações em finalizações.
  - **Goleiros**: Maximizar defesas difíceis realizadas e minimizar o número de gols sofridos.

Uma abordagem possível para considerar estas opções seria o uso de uma técnicas **multi-objetivo**, onde diferentes critérios são ponderados simultaneamente. Métodos comuns para lidar com problemas de múltiplos objetivos incluem:

- **Goal Programming**: Nesse método, define-se uma meta para cada objetivo de maneira hierárquicaÇ
    *  Escolhe-se o critério prioritário, otimiza-se o problema com tal critério, 
    *  Estabelece-se uma meta mínima a ser satisfeita para aquele critério como uma nova restrição e 
    *  busca-se otimizar baseando-se na meta seguinte. 

A solução do problema busca satisfazer a todas as metas. Por exemplo: escolhem-se os jogadores de ataque com maior média de "scout" (uma medida que inclui várias características desejáveis nos atacantes). Estabelecido a maior pontuação "scout" dos 3 jogadores de ataque (por exemplo, 27 pontos), determina-se uma nova restrição no problema que estabelece que a pontuação "scout" mínima para os 3 jogadores de ataque deve ser de 18 pontos.  Com a nova restrição, estabelece-se uma nova função objetivo: escolher os 3 jogadores de ataque maximizando o número médio de gols sofridos pelas equipes contra as quais eles (os atacantes) irão atuar naquela rodada. 
  
- **Função Ponderada**: Aqui, as diferentes características desejadas (como desarmes, gols, assistências, etc.) são combinadas em uma única função objetivo, atribuindo pesos para cada critério de acordo com sua importância. Isso permite que múltiplos aspectos do desempenho dos jogadores sejam levados em conta de forma equilibrada. Usando o mesmo exemplo anterior, poderia se estabelecer um peso 0.75 para os pontos scout e 0.25 para o número médio de gols sofridos pelas equipes adversárias.

Essas técnicas de otimização multi-objetivo podem ser aplicadas para fornecer soluções mais flexíveis e adaptadas a necessidades específicas do time-Cartola FC naquela rodada específica, considerando tanto o histórico dos jogadores quanto o histórico das equipes contra as quais eles irão atuar.  Além disso, pode permitir a inclusão de critérios qualitativos adicionais para a formação da equipe (número de expulsões, por exemplo).


## Modelo AMPL

Definições do conjunto de jogadores e parâmetros:

- **JOGADORES**: Conjunto de todos os jogadores disponíveis.
- **POSICOES**: Conjunto das posições possíveis no time.

### Parâmetros

- **media_j**: Média de pontuação do jogador **j**.
- **preco_j**: Preço do jogador **j**.
- **posicao_j**: Posição do jogador **j** (pode ser Goleiro, Zagueiro, Lateral, Meia, Atacante). Cada jogador tem apenas uma posição associada a ele.
- **orcamento**: Orçamento máximo disponível para a seleção dos jogadores.
- **num_jogadores_total**: Número total de jogadores que devem ser selecionados para o time. No caso, sempre igual a 11.
- **num_goleiros**: Número de goleiros necessários no time. No caso, sempre igual a 1.
- **num_zagueiros**: Número de zagueiros necessários no time. Nesse...
- **num_laterais**: Número de laterais necessários no time.
- **num_meias**: Número de meias necessários no time. Nesse...
- **num_atacantes**: Número de atacantes necessários no time. Nesse...





### Variáveis de Decisão

- \( x_j \): Variável binária que é 1 se o jogador \( j \) for escolhido, 0 caso contrário.

### Função Objetivo


Maximizar a média total dos jogadores selecionados:

$$
\text{Maximizar} \quad \sum_{j \in \text{JOGADORES}} \text{media\_ajustada}_j \cdot x_j
$$

### Restrições

1. **Restrição de Orçamento**: O custo total dos jogadores selecionados não deve exceder o orçamento disponível:

$$
\sum_{j \in \text{JOGADORES}} \text{preco}_j \cdot x_j \leq \text{orcamento}
$$

2. **Restrição do Número Total de Jogadores**: Exatamente 11 jogadores devem ser selecionados:

$$
\sum_{j \in \text{JOGADORES}} x_j = \text{num\_jogadores\_total}
$$

3. **Restrições por Posição**:

- **Goleiros**:

$$
\sum_{j \in \text{JOGADORES}, \, \text{posicao}_j = \text{"Goleiro"}} x_j = \text{num\_goleiros}
$$

- **Zagueiros**:

$$
\sum_{j \in \text{JOGADORES}, \, \text{posicao}_j = \text{"Zagueiro"}} x_j = \text{num\_zagueiros}
$$

- **Laterais**:

$$
\sum_{j \in \text{JOGADORES}, \, \text{posicao}_j = \text{"Lateral"}} x_j = \text{num\_laterais}
$$

- **Meias**:

$$
\sum_{j \in \text{JOGADORES}, \, \text{posicao}_j = \text{"Meia"}} x_j = \text{num\_meias}
$$

- **Atacantes**:

$$
\sum_{j \in \text{JOGADORES}, \, \text{posicao}_j = \text{"Atacante"}} x_j = \text{num\_atacantes}
$$

### Solução

Executamos o modelo de otimização para encontrar a seleção ótima de jogadores:

```ampl
solve;
display x;


## Referências ao Artigo
O modelo de otimização apresentado se baseia em conceitos descritos no artigo "Sports Analytics and Optimization for Team Formation Problems" de Megan L. Muniz. O artigo discute a formação de equipes otimizadas através da aplicação de técnicas de programação linear e de análise de dados em esportes. Em particular, a metodologia de otimização multiobjetivo e a consideração de restrições específicas para posições dos jogadores foram adaptadas para o contexto do futebol, conforme ilustrado no trecho a seguir:

"The ultimate aim is to develop a methodology to assist sports team executives (general managers and staff) in their team-building decision-making process, which includes a proposed solution methodology to improve bounds in an exact method." - Megan L. Muniz, Sports Analytics and Optimization for Team Formation Problems​(Muniz_mines_0052E_12420).

Esta abordagem foi adaptada para o contexto do futebol com restrições de posição e orçamento, que são comuns na gestão de equipes em esportes de fantasia como o Cartola FC.

## Programa


### 1. Instalações

In [2]:
%pip install gurobipy
%pip install requests
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 2. Importação de bibliotecas

In [3]:
import requests
import json as json
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

### 3. Requisição dos dados

#### 3.1. Requisição dos dados das partidas da rodada

In [4]:
url_partidas = "https://api.cartolafc.globo.com/partidas"
resposta_partidas = requests.get(url_partidas)
result_json = json.loads(resposta_partidas.content)

times = {
    time['nome'] : time['id']  for time in result_json['clubes'].values()
}

partidas = [
    {
        'time_casa': partida['clube_casa_id'],
        'time_casa_posicao': partida['clube_casa_posicao'],
        'time_visitante': partida['clube_visitante_id'],
        'time_visitante_posicao': partida['clube_visitante_posicao'],
    }
    for partida in result_json['partidas']
]

print(partidas)

[{'time_casa': 275, 'time_casa_posicao': 2, 'time_visitante': 282, 'time_visitante_posicao': 9}, {'time_casa': 263, 'time_casa_posicao': 1, 'time_visitante': 284, 'time_visitante_posicao': 14}, {'time_casa': 286, 'time_casa_posicao': 12, 'time_visitante': 280, 'time_visitante_posicao': 11}, {'time_casa': 276, 'time_casa_posicao': 5, 'time_visitante': 264, 'time_visitante_posicao': 17}, {'time_casa': 356, 'time_casa_posicao': 3, 'time_visitante': 1371, 'time_visitante_posicao': 19}, {'time_casa': 373, 'time_casa_posicao': 20, 'time_visitante': 266, 'time_visitante_posicao': 18}, {'time_casa': 285, 'time_casa_posicao': 8, 'time_visitante': 287, 'time_visitante_posicao': 16}, {'time_casa': 283, 'time_casa_posicao': 7, 'time_visitante': 267, 'time_visitante_posicao': 10}, {'time_casa': 265, 'time_casa_posicao': 6, 'time_visitante': 288, 'time_visitante_posicao': 13}, {'time_casa': 262, 'time_casa_posicao': 4, 'time_visitante': 293, 'time_visitante_posicao': 15}]


#### 3.2. Requisição dos dados da classificação atual do 'Brasileirão'

In [5]:

url_table = "https://api.football-data.org/v4/competitions/BSA/standings?season=2024"

mapeamento_nomes = {
    'Botafogo': 'Botafogo',
    'Palmeiras': 'Palmeiras',
    'Fortaleza': 'Fortaleza',
    'Flamengo': 'Flamengo',
    'São Paulo': 'São Paulo',
    'Bahia': 'Bahia',
    'Cruzeiro': 'Cruzeiro',
    'Internacional': 'Internacional',
    'Vasco da Gama': 'Vasco',
    'Mineiro': 'Atlético-MG',
    'Juventude': 'Juventude',
    'Bragantino': 'Bragantino',
    'Paranaense': 'Athlético-PR',
    'Grêmio': 'Grêmio',
    'Corinthians': 'Corinthians',
    'Criciúma': 'Criciúma',
    'Vitória': 'Vitória',
    'Fluminense': 'Fluminense',
    'Cuiabá EC': 'Cuiabá',
    'AC Goianiense': 'Atlético-GO'
}


headers = {
  'X-Auth-Token': 'cede0c34a4a54ca4a4bfe9bdd942b518',
}

response = requests.request("GET", url_table, headers=headers)

table = response.json()['standings'][0]['table']


filtered_table = [
  {
    'id': times[mapeamento_nomes[team['team']['shortName']]],
    'posicao': team['position'],
    'time': mapeamento_nomes[team['team']['shortName']],
    'pontos': team['points'],
    'vitorias': team['won'],
    'empates': team['draw'],
    'derrotas': team['lost'],
    'gols_pro': team['goalsFor'],
    'gols_contra': team['goalsAgainst'],
    'saldo_gols': team['goalDifference']
  }
  for team in table
]

df_table = pd.DataFrame(filtered_table)

print(df_table)



      id  posicao           time  pontos  vitorias  empates  derrotas  \
0    263        1       Botafogo      56        17        5         5   
1    275        2      Palmeiras      53        16        5         6   
2    356        3      Fortaleza      52        15        7         5   
3    262        4       Flamengo      45        13        6         7   
4    276        5      São Paulo      44        13        5         9   
5    285        6  Internacional      42        11        9         6   
6    265        7          Bahia      42        12        6         9   
7    283        8       Cruzeiro      42        12        6         9   
8    282        9    Atlético-MG      36         9        9         7   
9    267       10          Vasco      35        10        5        11   
10   280       11     Bragantino      32         8        8        11   
11   286       12      Juventude      32         8        8        11   
12   288       13       Criciúma      32         8 

#### 3.3 Requisição dos dados dos atletas do Cartola

In [6]:
# Passo 1: Obter dados da API do Cartola FC
url_atletas = "https://api.cartolafc.globo.com/atletas/mercado"
resposta_atletas = requests.get(url_atletas)
result_json = json.loads(resposta_atletas.content)

atletas_filter = []

for atleta in result_json['atletas']:
    time = atleta['clube_id']
    
    if atleta['status_id'] == 7:  # 7 indica que o jogador está "Provável"
        atletas_filter.append({
            'id': atleta['atleta_id'],
            'apelido': atleta['apelido'],
            'posicao': result_json['posicoes'][str(atleta['posicao_id'])]['nome'],
            'preco': atleta['preco_num'],
            'media': atleta['media_num'],
            'time': atleta['clube_id'],
        })


ids = [atleta['id'] for atleta in atletas_filter]
medias = {atleta['id']: atleta['media'] for atleta in atletas_filter} # iremos modificar isso
precos = {atleta['id']: atleta['preco'] for atleta in atletas_filter}
posicoes = {atleta['id']: atleta['posicao'] for atleta in atletas_filter}
apelidos = {atleta['id']: atleta['apelido'] for atleta in atletas_filter}

### 4. Colocando os dados dos times em consideração

#### 4.1 Criando um dicionario para mapear cada time ao seu adversario

In [7]:
adversarios = {}

for partida in partidas:
    time_casa = partida['time_casa']
    time_visitante = partida['time_visitante']
    adversarios[time_casa] = time_visitante
    adversarios[time_visitante] = time_casa
    
print(adversarios)

{275: 282, 282: 275, 263: 284, 284: 263, 286: 280, 280: 286, 276: 264, 264: 276, 356: 1371, 1371: 356, 373: 266, 266: 373, 285: 287, 287: 285, 283: 267, 267: 283, 265: 288, 288: 265, 262: 293, 293: 262}


#### 4.2 Adicionando os dados do time adversário de cada jogador

In [8]:
estatisticas_time = df_table.set_index('id').to_dict(orient='index')


for atleta in atletas_filter:
    time = atleta['time']
    atleta['adversario'] = adversarios[time]
    
    estatistica_adversario = estatisticas_time[atleta['adversario']]  
    atleta['adversario_media_gols_contra'] = estatistica_adversario['gols_contra'] / (estatistica_adversario['vitorias'] + estatistica_adversario['derrotas'] + estatistica_adversario['empates'])
    atleta['adversario_media_gols_pro'] = estatistica_adversario['gols_pro'] / (estatistica_adversario['vitorias'] + estatistica_adversario['derrotas'] + estatistica_adversario['empates'])

    
    estatisticas_clube = estatisticas_time[time]
    atleta['clube_media_gols_pro'] = estatisticas_clube['gols_pro'] / (estatisticas_clube['vitorias'] + estatisticas_clube['derrotas'] + estatisticas_clube['empates'])
    atleta['clube_media_gols_contra'] = estatisticas_clube['gols_contra'] / (estatisticas_clube['vitorias'] + estatisticas_clube['derrotas'] + estatisticas_clube['empates'])
    
print(atletas_filter)
    
    

[{'id': 102561, 'apelido': 'Mastriani', 'posicao': 'Atacante', 'preco': 6.75, 'media': 3.83, 'time': 293, 'adversario': 262, 'adversario_media_gols_contra': 1.2307692307692308, 'adversario_media_gols_pro': 1.6153846153846154, 'clube_media_gols_pro': 1.08, 'clube_media_gols_contra': 1.16}, {'id': 87008, 'apelido': 'Léo Jardim', 'posicao': 'Goleiro', 'preco': 6.07, 'media': 3.62, 'time': 267, 'adversario': 283, 'adversario_media_gols_contra': 1.0, 'adversario_media_gols_pro': 1.2592592592592593, 'clube_media_gols_pro': 1.1538461538461537, 'clube_media_gols_contra': 1.3846153846153846}, {'id': 72294, 'apelido': 'Everson', 'posicao': 'Goleiro', 'preco': 8.08, 'media': 2.43, 'time': 282, 'adversario': 275, 'adversario_media_gols_contra': 0.7037037037037037, 'adversario_media_gols_pro': 1.6296296296296295, 'clube_media_gols_pro': 1.4, 'clube_media_gols_contra': 1.44}, {'id': 39148, 'apelido': 'Hulk', 'posicao': 'Atacante', 'preco': 15.04, 'media': 8.13, 'time': 282, 'adversario': 275, 'adver

#### 4.4 Calculando as médias da liga

In [9]:

df_table['jogos'] = df_table['vitorias'] + df_table['empates'] + df_table['derrotas']

df_table['gols_pro_por_jogo'] = df_table['gols_pro'] / df_table['jogos']
df_table['gols_contra_por_jogo'] = df_table['gols_contra'] / df_table['jogos']


media_gols_pro_liga = df_table['gols_pro_por_jogo'].mean()
media_gols_contra_liga = df_table['gols_contra_por_jogo'].mean()

print(media_gols_pro_liga)
print(media_gols_contra_liga)

1.218088319088319
1.2196011396011397


In [10]:
medias_atacantes_meias = [atleta['media'] for atleta in atletas_filter if atleta['posicao'] in ['Atacante', 'Meia']]
clube_gols_pro = [atleta['clube_media_gols_pro'] for atleta in atletas_filter if atleta['posicao'] in ['Atacante', 'Meia']]
adversario_gols_contra = [atleta['adversario_media_gols_contra'] for atleta in atletas_filter if atleta['posicao'] in ['Atacante', 'Meia']]


media_min_atk = min(medias_atacantes_meias)
media_max_atk = max(medias_atacantes_meias)

print(media_max_atk)
print(media_min_atk)

clube_gols_pro_min = min(clube_gols_pro)
clube_gols_pro_max = max(clube_gols_pro)

adversario_gols_contra_min = min(adversario_gols_contra)
adversario_gols_contra_max = max(adversario_gols_contra)

9.5
0.6


In [11]:
media_defensores = [atleta['media'] for atleta in atletas_filter if atleta['posicao'] in ['Zagueiro', 'Lateral', 'Goleiro']]

media_min = min(media_defensores)
media_max = max(media_defensores)

clube_gols_contra_defensores = [atleta['clube_media_gols_contra'] for atleta in atletas_filter if atleta['posicao'] in ['Zagueiro', 'Goleiro', 'Lateral']]
clube_gols_contra_min = min(clube_gols_contra_defensores)
clube_gols_contra_max = max(clube_gols_contra_defensores)

adversario_gols_pro_defensores = [atleta['adversario_media_gols_pro'] for atleta in atletas_filter if atleta['posicao'] in ['Zagueiro', 'Goleiro', 'Lateral']]
adversario_gols_pro_min = min(adversario_gols_pro_defensores)
adversario_gols_pro_max = max(adversario_gols_pro_defensores)

#### 4.3 Calculando o fator de ajuste da média de cada jogador

In [12]:
meta_media_jogador_atk = 6.5
meta_gols_pro_clube_atk = 1.5
meta_gols_contra_adv_atk = 1.4


meta_media_normalizada = (meta_media_jogador_atk - media_min_atk) / (media_max_atk - media_min_atk)
meta_gols_pro_normalizada = (meta_gols_pro_clube_atk - clube_gols_pro_min) / (clube_gols_pro_max - clube_gols_pro_min)
meta_gols_contra_normalizada = (meta_gols_contra_adv_atk - adversario_gols_contra_min) / (adversario_gols_contra_max - adversario_gols_contra_min)


for atleta in atletas_filter:
    atleta['media_ajustada'] = atleta['media']
    if atleta['posicao'] in ['Atacante', 'Meia']:
        media_normalizada = (atleta['media'] - media_min_atk) / (media_max - media_min_atk)
        clube_gols_pro_normalizada = (atleta['clube_media_gols_pro'] - clube_gols_pro_min) / (clube_gols_pro_max - clube_gols_pro_min)
        adversario_gols_contra_normalizada = (atleta['adversario_media_gols_contra'] - adversario_gols_contra_min) / (adversario_gols_contra_max - adversario_gols_contra_min)
        
        desvio_negativo_media = max(0, meta_media_normalizada - media_normalizada)  # falta
        desvio_positivo_media = max(0, media_normalizada - meta_media_normalizada)  # excesso
        
   
        desvio_negativo_gols_pro = max(0, meta_gols_pro_normalizada - clube_gols_pro_normalizada)
        desvio_positivo_gols_pro = max(0, clube_gols_pro_normalizada - meta_gols_pro_normalizada)

        desvio_negativo_gols_contra = max(0, meta_gols_contra_normalizada - adversario_gols_contra_normalizada)
        desvio_positivo_gols_contra = max(0, adversario_gols_contra_normalizada - meta_gols_contra_normalizada)
       
        peso_media = 0.6  
        peso_gols_pro = 0.2  
        peso_gols_contra = 0.2  

      
        fator_media = peso_media * (desvio_negativo_media ) 
        fator_gols_pro = peso_gols_pro * (desvio_negativo_gols_pro )
        fator_gols_contra = peso_gols_contra * (desvio_negativo_gols_contra)
        
        fator_total = fator_media + fator_gols_pro + fator_gols_contra
        
        media_ajustada = 0.5 / fator_total

        atleta['media_ajustada'] = media_ajustada * (media_max - media_min_atk) + media_min_atk

        # Imprimir para verificar
        print(f"Jogador {atleta['apelido']}:")
        print(f"Média original: {atleta['media']} -> Média Normalizada: {media_normalizada} -> Média ajustada: {atleta['media_ajustada']}")
        print(f"Media de gols do clube: {atleta['clube_media_gols_pro']}, Media de gols sofridos do adversario: {atleta['adversario_media_gols_contra']}")
        print(f"Desvio da meta de média: {desvio_negativo_media }")
        print(f"Desvio da meta de gols pro do time: {desvio_negativo_gols_pro}")
        print(f"Desvio da meta de gols contra adversario: {desvio_negativo_gols_contra }")
        print(f"Fator media: {fator_media}, Fator gols pro: {fator_gols_pro}, Fator gols contra: {fator_gols_contra}, Fator total: {fator_media + fator_gols_pro + fator_gols_contra}")
        
        print("---------------------------------------------------------------")
        
        
    if atleta['posicao'] in ['Zagueiro', 'Goleiro', 'Lateral']:
        # Normalizando
        media_normalizada = (atleta['media'] - media_min) / (media_max - media_min)
        
        clube_gols_contra_normalizado = 1/atleta['clube_media_gols_contra']
        
        adversario_gols_pro_normalizado = 1/ atleta['adversario_media_gols_pro']
        
        peso_media = 0.4
        peso_clube = 0.3
        peso_adversario = 0.3
        
        media_ajustada = (
            peso_media * media_normalizada +
            peso_clube * clube_gols_contra_normalizado +
            peso_adversario * adversario_gols_pro_normalizado
        )
        
        # Escalar de volta para a escala original da média
        # atleta['media_ajustada'] = media_ajustada * (media_max - media_min) + media_min
        
        # print(f"Jogador {atleta['apelido']}:")
        # print(f"Media_normalizada: {media_normalizada}, Clube_gols_contra_normalizado: {clube_gols_contra_normalizado}, Adversario_gols_pro_normalizado: {adversario_gols_pro_normalizado}")
        # print(f"Media antes:{atleta['media']}, Media depois:{atleta['media_ajustada']}")
        # print("---------------------------------------------------------------")

Jogador Mastriani:
Média original: 3.83 -> Média Normalizada: 0.4205729166666667 -> Média ajustada: 14.755273650571523
Media de gols do clube: 1.08, Media de gols sofridos do adversario: 1.2307692307692308
Desvio da meta de média: 0.24234843164794012
Desvio da meta de gols pro do time: 0.45359999999999995
Desvio da meta de gols contra adversario: 0.17573964497041406
Fator media: 0.14540905898876408, Fator gols pro: 0.09072, Fator gols contra: 0.03514792899408281, Fator total: 0.2712769879828469
---------------------------------------------------------------
Jogador Hulk:
Média original: 8.13 -> Média Normalizada: 0.9804687500000002 -> Média ajustada: 23.70255460940393
Media de gols do clube: 1.4, Media de gols sofridos do adversario: 0.7037037037037037
Desvio da meta de média: 0
Desvio da meta de gols pro do time: 0.1080000000000001
Desvio da meta de gols contra adversario: 0.723076923076923
Fator media: 0.0, Fator gols pro: 0.021600000000000022, Fator gols contra: 0.1446153846153846, 

### 5. Definição dos modelos matemáticos

#### Definição do Modelo
Baseando-se nas técnicas de otimização discutidas por Muniz (2024), utilizamos a programação linear binária para modelar o problema de seleção de jogadores. Cada jogador é representado por uma variável binária que indica se ele foi selecionado para a equipe. O objetivo é maximizar a média de pontuação dos jogadores selecionados, respeitando o orçamento e as restrições de composição da equipe.

In [13]:
# Definição das restrições
orcamento = 100.0  # Orçamento máximo
num_jogadores_total = 11
num_goleiros = 1
num_zagueiros = 2
num_laterais = 2
num_meias = 3
num_atacantes = 3

# Inicializa o modelo
model = gp.Model("time_futebol_cartola")


Restricted license - for non-production use only - expires 2025-11-24


In [14]:
# Adiciona as variáveis de decisão
x = model.addVars(ids, vtype=GRB.BINARY, name="x")

# Função objetivo: maximizar a média ajustada do time
model.setObjective(gp.quicksum(atleta['media_ajustada'] * x[atleta['id']] for atleta in atletas_filter), GRB.MAXIMIZE)

# Restrição de orçamento máximo
model.addConstr(gp.quicksum(precos[atleta['id']] * x[atleta['id']] for atleta in atletas_filter) <= orcamento, "Orcamento")

# Restrição para número total de jogadores
model.addConstr(gp.quicksum(x[atleta['id']] for atleta in atletas_filter) == num_jogadores_total, "Total_Jogadores")

# Restrições para cada posição
model.addConstr(gp.quicksum(x[atleta['id']] for atleta in atletas_filter if atleta['posicao'] == 'Goleiro') == num_goleiros, "Goleiros")
model.addConstr(gp.quicksum(x[atleta['id']] for atleta in atletas_filter if atleta['posicao'] == 'Zagueiro') == num_zagueiros, "Zagueiros")
model.addConstr(gp.quicksum(x[atleta['id']] for atleta in atletas_filter if atleta['posicao'] == 'Lateral') == num_laterais, "Laterais")
model.addConstr(gp.quicksum(x[atleta['id']] for atleta in atletas_filter if atleta['posicao'] == 'Meia') == num_meias, "Meias")
model.addConstr(gp.quicksum(x[atleta['id']] for atleta in atletas_filter if atleta['posicao'] == 'Atacante') == num_atacantes, "Atacantes")

<gurobi.Constr *Awaiting Model Update*>

### 6. Solução
#### Solução do Modelo
O modelo é resolvido utilizando o solver Gurobi, que é eficiente em lidar com problemas de otimização combinatória de grande escala. Uma solução ótima fornece a melhor formação de equipe possível dentro das restrições especificadas.

In [15]:
# Resolve o modelo
model.optimize()

# Verifica o status da solução
if model.status == GRB.OPTIMAL:
    print("Solução Ótima Encontrada")
    for atleta in atletas_filter:
        if x[atleta['id']].x > 0.5:  # Verifica se o jogador foi selecionado
            print(f"Jogador: {atleta['apelido']}, Posição: {atleta['posicao']}, Média Ajustada: {atleta['media_ajustada']}, Preço: {atleta['preco']}")
else:
    print("Não foi possível encontrar uma solução ótima.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19044.2))

CPU model: AMD Ryzen 5 3500X 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads

Optimize a model with 7 rows, 231 columns and 673 nonzeros
Model fingerprint: 0x13a3837b
Variable types: 0 continuous, 231 integer (231 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [7e-01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 271.7603617
Presolve removed 1 rows and 31 columns
Presolve time: 0.02s
Presolved: 6 rows, 200 columns, 395 nonzeros
Variable types: 0 continuous, 200 integer (200 binary)
Found heuristic solution: objective 961.7746747

Root relaxation: objective 1.083120e+03, 20 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Inc

### Resultados e Discussão

Os resultados obtidos são analisados para avaliar a eficácia da equipe formada com base nos critérios de otimização escolhidos. A equipe ideal foi composta de forma a maximizar a pontuação média dos jogadores, respeitando as restrições de orçamento e posição (formação 4-3-3). Essa abordagem garantiu uma seleção balanceada de jogadores com alto potencial de pontuação, dentro das limitações impostas pelo jogo *Cartola FC*.

Uma análise de sensibilidade foi conduzida para entender como variações nos parâmetros de entrada influenciam a composição final da equipe. Como descrito por Muniz (2024), a **análise de sensibilidade** é uma ferramenta poderosa para avaliar a robustez das soluções de otimização. No contexto deste trabalho, a análise de sensibilidade permite observar o impacto de alterações em variáveis-chave, como:

- **Orçamento**: Ao alterar o limite de orçamento disponível para a formação da equipe, é possível verificar como a composição da equipe muda. Equipes com maior orçamento tendem a incluir jogadores com pontuação média mais alta, enquanto equipes com orçamentos menores podem exigir uma escolha mais estratégica, priorizando jogadores com bom custo-benefício.

- **Restrições de posição**: Alterar as restrições de posição (por exemplo, testar formações diferentes como 3-5-2 ou 4-4-2) permite analisar como a distribuição de jogadores entre as posições influencia o desempenho geral da equipe. Formações mais ofensivas, por exemplo, podem resultar em maior dependência de atacantes com alta pontuação, enquanto formações defensivas podem valorizar jogadores com bom desempenho em desarmes e defesas.

- **Critérios de pontuação**: Alterar o foco da otimização para características específicas de desempenho, como priorizar assistências no meio-campo ou defesas difíceis para goleiros, também afeta diretamente a composição final da equipe. Ao realizar essa análise, podemos entender melhor como a escolha de diferentes funções objetivo influencia a estrutura do time e suas chances de sucesso.

A **análise de sensibilidade** também é crucial para identificar possíveis limitações do modelo de otimização. Por exemplo, equipes formadas com orçamentos muito baixos podem não ser capazes de competir de maneira eficaz em relação a equipes com orçamentos maiores, mesmo otimizando suas escolhas. Além disso, mudanças nas regras do jogo ou nas métricas de pontuação podem impactar a eficácia da equipe formada.

Essa abordagem de análise sistemática fornece uma visão clara sobre a flexibilidade e os trade-offs envolvidos na formação de equipes, permitindo que ajustes sejam feitos conforme necessário para maximizar o desempenho em diferentes cenários.


# Conclusão
Este notebook demonstrou como técnicas avançadas de otimização podem ser aplicadas ao problema de formação de equipes no contexto do Cartola FC. A utilização de programação matemática permite uma abordagem sistemática e eficiente para selecionar os melhores jogadores, considerando múltiplos critérios e restrições. Este método pode ser adaptado e expandido para outros contextos esportivos, fornecendo uma ferramenta poderosa para analistas e gestores esportivos.



# Fórmulas de Pontuação por Posição no Cartola
### Este é um exemplo de modelo completo, porém não é o que foi implementando em nossa abordagem.

### Para Jogadores Ofensivos:
score_final = x1 * media + x2 * finalizacoes_concluidas + x3 * previsao_de_gol_ou_assistencia

- **Finalizações Concluídas**:
  finalizacoes_concluidas = y1 * finalizacoes_no_gol + y2 * finalizacoes_para_fora
  
- **Previsão de Gol ou Assistência**:
  previsao_de_gol_ou_assistencia = p1 * qtd_gols_jogador_marcou + p2 * qtd_gols_time_marcou + p3 * qtd_gols_adversario_tomou + p4 * qtd_assistencia_jogador_deu

**Pesos**:
- x1 = 0.5
- x2 = 0.2
- x3 = 0.3
- y1 = 0.8
- y2 = 0.2
- p1 = 0.4
- p2 = 0.2
- p3 = 0.3
- p4 = 0.1

### Para Jogadores de Meio:
score_final = x1 * media + x2 * finalizacoes_concluidas + x3 * previsao_de_gol_ou_assistencia + x4 * qtd_desarmes

**Finalizações Concluídas**:
finalizacoes_concluidas = y1 * finalizacoes_no_gol + y2 * finalizacoes_para_fora

**Previsão de Gol ou Assistência**:
previsao_de_gol_ou_assistencia = p1 * qtd_gols_jogador_marcou + p2 * qtd_gols_time_marcou + p3 * qtd_gols_adversario_tomou + p4 * qtd_assistencia_jogador_deu

**Pesos**:
- x1 = 0.3
- x2 = 0.2
- x3 = 0.3
- x4 = 0.2
- y1 = 0.8
- y2 = 0.2
- p1 = 0.4
- p2 = 0.2
- p3 = 0.3
- p4 = 0.1

### Para Zagueiros e Laterais:
score_final = x1 * media + x2 * finalizacoes_concluidas + x3 * previsao_de_gol_ou_assistencia + x4 * qtd_desarmes + x5 * previsao_de_saldo_de_gol

**Finalizações Concluídas**:
finalizacoes_concluidas = y1 * finalizacoes_no_gol + y2 * finalizacoes_para_fora

**Previsão de Saldo de Gol**:
previsao_de_saldo_de_gol = k1 * qtd_jogos_time_nao_tomou_gol + k2 * qtd_jogos_jogador_nao_tomou_gol + k3 * qtd_jogos_adversario_nao_fez_gol

**Previsão de Gol ou Assistência**:
previsao_de_gol_ou_assistencia = p1 * qtd_gols_jogador_marcou + p2 * qtd_gols_time_marcou + p3 * qtd_gols_adversario_tomou + p4 * qtd_assistencia_jogador_deu

**Pesos**:
- x1 = 0.3
- x2 = 0.1
- x3 = 0.1
- x4 = 0.2
- x5 = 0.3
- y1 = 0.8
- y2 = 0.2
- k1 = 0.4
- k2 = 0.4
- k3 = 0.2
- p1 = 0.2
- p2 = 0.3
- p3 = 0.3
- p4 = 0.2

### Para Goleiros:
score_final = x1 * media + x2 * qtd_defesas + x3 * previsao_de_saldo_de_gol + x4 * qtd_gols_sofridos + x5 * qtd_penaltis_defendidos

**Previsão de Saldo de Gol**:
previsao_de_saldo_de_gol = k1 * qtd_jogos_time_nao_tomou_gol + k2 * qtd_jogos_goleiro_nao_tomou_gol + k3 * qtd_jogos_adversario_nao_fez_gol

**Pesos**:
- x1 = 0.3 (peso da média do goleiro)
- x2 = 0.3 (peso das defesas feitas)
- x3 = 0.2 (peso da previsão de saldo de gol)
- x4 = 0.1 (peso para gols sofridos)
- x5 = 0.3 (peso dos pênaltis defendidos)
- k1 = 0.4 (peso de jogos em que o time não tomou gol)
- k2 = 0.4 (peso de jogos em que o goleiro não tomou gol)
- k3 = 0.2 (peso de jogos em que o adversário não fez gol)

---
